In [1]:
import os
os.environ['https_proxy'] = "http://proxy.hcm.fpt.vn:80/"
os.environ['http_proxy'] = "http://proxy.hcm.fpt.vn:80/"
os.environ["WANDB_DISABLED"] = "true"

In [2]:
!git config --global http.proxy "http://proxy.hcm.fpt.vn:80"
!git config --global http.https://git.bigdata.local.proxy """
!export http_proxy=proxy.hcm.fpt.vn
!export https_proxy=proxy.hcm.fpt.vn

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [3]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [4]:
# Load the 'tapaco' dataset in English
dataset = load_dataset("tapaco", "en")

# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

Found cached dataset tapaco (/home/sinhns2/.cache/huggingface/datasets/tapaco/en/1.0.0/71d200534b520a174927a8f0479c06220a0a6fb5201a84ebfce19006c6354698)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
df

,paraphrase_set_id,sentence_id,paraphrase,lists,tags,language
0,1,416554,I ate the cheese.,"[907, 4000, 6677, 7361, 7415]",[],en
1,1,2481696,I eat cheese.,[907],[],en
2,1,2721028,I'm eating a yogurt.,"[992, 3800]",[],en
3,1,3010891,I'm eating cheese.,[6905],[],en
4,1,4129977,I'm having some cheese.,[6905],[],en
...,...,...,...,...,...,...
158048,843414,941276,All of you are welcome to attend my party alon...,[649],[],en
158049,844191,962048,I wouldn't like being a judge.,"[907, 4000, 7414]",[8 syllables],en
158050,844191,962049,I wouldn't want to be a judge.,"[907, 4000, 7414]",[8 syllables],en
158051,844308,988514,To attend is important!,[649],[],en


In [7]:
def generate_tapaco_paraphrase_dataset(dataset, out_file):
    dataset_df = dataset[["paraphrase", "paraphrase_set_id"]]
    non_single_labels = (
        dataset_df["paraphrase_set_id"]
        .value_counts()[dataset_df["paraphrase_set_id"].value_counts() > 1]
        .index.tolist()
    )
    tapaco_df_sorted = dataset_df.loc[
        dataset_df["paraphrase_set_id"].isin(non_single_labels)
    ]
    tapaco_paraphrases_dataset = []

    for paraphrase_set_id in tqdm(tapaco_df_sorted["paraphrase_set_id"].unique()):
        id_wise_paraphrases = tapaco_df_sorted[
            tapaco_df_sorted["paraphrase_set_id"] == paraphrase_set_id
        ]
        len_id_wise_paraphrases = (
            id_wise_paraphrases.shape[0]
            if id_wise_paraphrases.shape[0] % 2 == 0
            else id_wise_paraphrases.shape[0] - 1
        )
        for ix in range(0, len_id_wise_paraphrases, 2):
            current_phrase = id_wise_paraphrases.iloc[ix][0]
            for count_ix in range(ix + 1, ix + 2):
                next_phrase = id_wise_paraphrases.iloc[ix + 1][0]
                tapaco_paraphrases_dataset.append([current_phrase, next_phrase])
    tapaco_paraphrases_dataset_df = pd.DataFrame(
        tapaco_paraphrases_dataset, columns=["Text", "Paraphrase"]
    )
    tapaco_paraphrases_dataset_df.to_csv(out_file, sep="\t", index=None)
    return tapaco_paraphrases_dataset_df

tapaco_paraphrases_dataset_df = generate_tapaco_paraphrase_dataset(df,"tapaco_paraphrases_dataset.csv")

In [4]:
tapaco_paraphrases_dataset_df = pd.read_csv("tapaco_paraphrases_dataset.csv", sep="\t")
tapaco_paraphrases_dataset_df

,Text,Paraphrase
0,I ate the cheese.,I eat cheese.
1,I'm eating a yogurt.,I'm eating cheese.
2,I'm having some cheese.,I eat some cheese.
3,It's Monday.,It is Monday today.
4,It's Monday today.,Today is Monday.
...,...,...
73513,What's your favorite horror movie?,What's your favorite scary movie?
73514,Globalise or die.,Globalize or die.
73515,All of you are welcome to bring your spouses t...,All of you are welcome to attend my party alon...
73516,I wouldn't like being a judge.,I wouldn't want to be a judge.


## Model Training

In [12]:
!pip install protobuf==3.20.1

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

2023-05-14 15:37:02.048433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 15:37:12.194018: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-14 15:37:12.194851: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-14 15:37:12.194887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [9]:
tapaco_paraphrases_dataset_df = pd.read_csv("tapaco_paraphrases_dataset.csv", sep="\t")
tapaco_paraphrases_dataset_df.columns = ["input_text","target_text"]
tapaco_paraphrases_dataset_df["prefix"] = "paraphrase"
train_data,val_data = train_test_split(tapaco_paraphrases_dataset_df,test_size=0.1)

In [10]:
val_data

,input_text,target_text,prefix
40775,We haven't seen him.,We haven't seen her.,paraphrase
8494,Anyone can do that.,Anyone can do it.,paraphrase
49418,We will probably arrive at Tokyo station at noon.,It's possible we'll arrive at the Tokyo statio...,paraphrase
45189,Why do I always need to struggle with such pro...,Why do I always have to put up with these things?,paraphrase
10993,This is the first time for me to read the Bible.,It’s the first time that I read the Bible.,paraphrase
...,...,...,...
45101,I've seen a couple of Kurosawa's films.,I have seen at least one Kurosawa film.,paraphrase
57843,"Although you rushed, you're not ready.","Even though you're quick, you're still not ready.",paraphrase
40235,You guys are Swedish.,You all are Swedish.,paraphrase
2120,I'm from out of town.,I'm new around here.,paraphrase


In [62]:
# Define data trainer
def tokenize_data(examples):
    inputs = [f"{prefix}: {input_text}" for prefix, input_text in zip(examples['prefix'], examples['input_text'])]
    targets = list(examples['target_text'])
    input_encodings = tokenizer.batch_encode_plus(inputs, max_length=512, truncation=True, padding=True)
    target_encodings = tokenizer.batch_encode_plus(targets, max_length=512, truncation=True, padding=True)
    input_ids = torch.tensor(input_encodings['input_ids'])
    input_mask = torch.tensor(input_encodings['attention_mask'])
    target_ids = torch.tensor(target_encodings['input_ids'])
    target_mask = torch.tensor(target_encodings['attention_mask'])
    return {'input_ids': input_ids, 'attention_mask': input_mask, 'labels': target_ids}

    
# Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(train_data)

# Tokenize the data
train_dataset = train_dataset.map(tokenize_data, batched=True)

  0%|          | 0/67 [00:00<?, ?ba/s]

In [63]:
# Convert DataFrame to Dataset
val_dataset = Dataset.from_pandas(val_data)

# Tokenize the data
val_dataset = val_dataset.map(tokenize_data, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

In [70]:
import torch
from torch.nn.utils.rnn import pad_sequence

def data_collator(data):
    input_ids = [torch.tensor(item["input_ids"], dtype=torch.long) for item in data]
    attention_mask = [torch.tensor(item["attention_mask"], dtype=torch.long) for item in data]
    padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    padded_attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    return {
        "input_ids": padded_input_ids,
        "attention_mask": padded_attention_mask
    }


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load and preprocess data
tapaco_paraphrases_dataset_df = pd.read_csv("tapaco_paraphrases_dataset.csv", sep="\t")
tapaco_paraphrases_dataset_df.columns = ["input_text", "target_text"]
tapaco_paraphrases_dataset_df["prefix"] = "paraphrase"
train_data, val_data = train_test_split(tapaco_paraphrases_dataset_df, test_size=0.1)

# Load T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # directory to save checkpoints and logs
    num_train_epochs=3,  # total number of training epochs
    per_device_train_batch_size=4,  # batch size per GPU
    per_device_eval_batch_size=4,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # weight decay coefficient for regularization
    logging_dir="./logs",  # directory to save training logs
    logging_steps=500,  # number of steps to save training logs
    evaluation_strategy="steps",  # evaluation strategy after steps or epochs
    eval_steps=500,  # evaluation steps
    save_total_limit=3,  # limit to number of checkpoints saved
    load_best_model_at_end=True,  # load the best model when training ends
    metric_for_best_model="eval_loss",  # metric to monitor for saving the best model
    greater_is_better=False,  # whether the metric for best model should be maximized or minimized
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()


/home/sinhns2/envs/python38/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


NameError: name 'train_dataset' is not defined

In [4]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Load the tokenizer and the pre-trained T5 model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Load the Tapaco paraphrase dataset
tapaco_paraphrases_dataset_df = pd.read_csv("tapaco_paraphrases_dataset.csv", sep="\t")
tapaco_paraphrases_dataset_df.columns = ["input_text","target_text"]

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(tapaco_paraphrases_dataset_df, test_size=0.1)

# Define the training and validation datasets using the Hugging Face Dataset class
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Define the function to preprocess the data
def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["target_text"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    with tokenizer.as_target_tokenizer():
        model_targets = tokenizer(targets, padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = model_targets["input_ids"]
    return model_inputs

# Preprocess the training and validation datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

2023-05-14 15:56:25.365238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 15:56:25.483789: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-14 15:56:28.056552: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-14 15:56:28.056630: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

  0%|          | 0/67 [00:00<?, ?ba/s]

/home/sinhns2/envs/python38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/8 [00:00<?, ?ba/s]

In [5]:
import wandb

wandb.init()
# ... calculate metrics, generate media
wandb.log({"accuracy": 0.9})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [13]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
)

# Set training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./similar_question_t5",
    evaluation_strategy="steps",
    save_steps=500,
    save_total_limit=5,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    num_train_epochs=50,
    predict_with_generate=True,
    logging_steps=1000,
    overwrite_output_dir=True,
)

# Create trainer and start training
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

# Save fine-tuned model
trainer.save_model("./fine_tuned_similar_question_generation_model")

using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


RuntimeError: false INTERNAL ASSERT FAILED at "../c10/cuda/CUDAGraphsC10Utils.h":73, please report a bug to PyTorch. Unknown CUDA graph CaptureStatus22094

In [13]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [5]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments 
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Load the tokenizer and the pre-trained T5 model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Load the Tapaco paraphrase dataset
tapaco_paraphrases_dataset_df = pd.read_csv("tapaco_paraphrases_dataset.csv", sep="\t")
tapaco_paraphrases_dataset_df.columns = ["input_text","target_text"]

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(tapaco_paraphrases_dataset_df, test_size=0.1)

# Define the training and validation datasets using the Hugging Face Dataset class
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Define the function to preprocess the data
def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["target_text"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    with tokenizer.as_target_tokenizer():
        model_targets = tokenizer(targets, padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = model_targets["input_ids"]
    return model_inputs

# Preprocess the training and validation datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    save_total_limit=5,
    learning_rate=1e-3,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000
)

# Define the trainer and train the model
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)
trainer.train()
trainer.save_model("./fine_tuned_similar_question_generation_model_t5_small_v2")
tokenizer.save_pretrained("./fine_tuned_similar_question_generation_model_t5_small_v2")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: input_text, target_text, __index_level_0__. If input_text, target_text, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/home/sinhns2/envs/python38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 66166
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, 

Step,Training Loss
1000,0.222600
2000,0.011500
3000,0.010800
4000,0.010700


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Configuration saved in ./results/checkpoint-500/generation_config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-2000] due to args.save_total_limit
/home/sinhns2/envs/python38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Configuration saved in ./results/checkpoint-1000/generation_config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-2500] due to args.save_total_limit
/home/sinhns2/envs/python38/lib/python3.8/site

('./fine_tuned_similar_question_generation_model_t5_small_v2/tokenizer_config.json',
 './fine_tuned_similar_question_generation_model_t5_small_v2/special_tokens_map.json',
 './fine_tuned_similar_question_generation_model_t5_small_v2/spiece.model',
 './fine_tuned_similar_question_generation_model_t5_small_v2/added_tokens.json')

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("./fine_tuned_similar_question_generation_model_t5_small")
tokenizer = T5Tokenizer.from_pretrained("./fine_tuned_similar_question_generation_model_t5_small")

loading configuration file ./fine_tuned_similar_question_generation_model_t5_small/config.json
Model config T5Config {
  "_name_or_path": "./fine_tuned_similar_question_generation_model_t5_small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
 

OSError: Can't load tokenizer for './fine_tuned_similar_question_generation_model_t5_small'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './fine_tuned_similar_question_generation_model_t5_small' is the correct path to a directory containing all relevant files for a T5Tokenizer tokenizer.

In [ ]:
load_tokenizer = 

Saving model checkpoint to ./fine_tuned_similar_question_generation_model_t5_small
Configuration saved in ./fine_tuned_similar_question_generation_model_t5_small/config.json
Configuration saved in ./fine_tuned_similar_question_generation_model_t5_small/generation_config.json
Model weights saved in ./fine_tuned_similar_question_generation_model_t5_small/pytorch_model.bin


### Continue 

In [73]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
# Load the Tapaco paraphrase dataset
tapaco_paraphrases_dataset_df = pd.read_csv("tapaco_paraphrases_dataset.csv", sep="\t")
tapaco_paraphrases_dataset_df.columns = ["input_text","target_text"]

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(tapaco_paraphrases_dataset_df, test_size=0.1)

# Define the training and validation datasets using the Hugging Face Dataset class
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

In [64]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model_test = T5ForConditionalGeneration.from_pretrained('./fine_tuned_similar_question_generation_model_t5_small_v2')
tokenizer_test = T5Tokenizer.from_pretrained('./fine_tuned_similar_question_generation_model_t5_small_v2')

2023-05-21 12:38:39.925236: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 12:38:40.037064: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-21 12:38:44.084097: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-21 12:38:44.084169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [70]:
text = "I'm ashamed of what I did."
inputs = tokenizer_test.encode(text, return_tensors="pt")
generated_ids = model_test.generate(inputs)
generated_text = tokenizer_test.decode(generated_ids.squeeze(), skip_special_tokens=True)
print(generated_text)

I'm ashamed of what I did.


### Evaluation

In [71]:
def generate_predictions(model, tokenizer, dataset):
    predictions = []
    for example in dataset:
        input_text = example["input_text"]
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        generated_ids = model.generate(input_ids)
        generated_text = tokenizer.decode(generated_ids.squeeze(), skip_special_tokens=True)
        predictions.append(generated_text)
    return predictions


In [74]:
first_10_rows = val_dataset.select(range(10))


In [75]:
first_10_rows

Dataset({
    features: ['input_text', 'target_text', '__index_level_0__'],
    num_rows: 10
})

In [76]:
val_predictions = generate_predictions(model_test, tokenizer_test,first_10_rows)

/home/sinhns2/envs/python38/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [77]:
val_targets = val_dataset["target_text"][:10]

In [78]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

bleu_score = corpus_bleu([[target] for target in val_targets], val_predictions)
print(f"BLEU score: {bleu_score}")


BLEU score: 0.5896977684739921


In [23]:
from sacrebleu.metrics import BLEU, CHRF, TER
bleu = BLEU()
bleu.corpus_score(val_targets, val_predictions)

BLEU = 1.44 10.4/1.2/0.7/0.5 (BP = 1.000 ratio = 6.857 hyp_len = 48 ref_len = 7)

In [24]:
bleu = BLEU()
bleu.corpus_score(val_targets, val_predictions)

BLEU = 0.76 1.6/0.9/0.6/0.4 (BP = 1.000 ratio = 6.300 hyp_len = 63 ref_len = 10)

In [25]:
bleu.get_signature()

nrefs:10|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1

In [26]:
chrf = CHRF()

In [27]:
chrf.corpus_score(val_targets, val_predictions)

chrF2 = 16.36

In [28]:
ter = TER()

In [79]:
ter.corpus_score(val_targets, val_predictions)

NameError: name 'ter' is not defined

In [30]:
for i in range(0,9):
    print(val_targets[i], val_predictions[i])

You want to watch a French film, right? Do you really want to see a French film?
I haven't touched anything. I didn't touch anything.
Let's go ask her. Let's go ask her.
Prices are rising. Prices are rising.
I can’t stand panties. I don't have any tolerance for cowards.
Tom told Mary that he didn't think Alice was beautiful. Tom told Mary that he didn't think Alice was beautiful.
I'm ashamed of what I did. I'm ashamed of my conduct.
Tom seemed like a good kid. Tom seemed to be a decent kid.
Tom sunbathed. Tom tanned.


In [80]:
for i in range(0,9):
    print(val_targets[i], val_predictions[i])

Tom is a terrible kisser. Tom is a lousy kisser.
They took off their clothes. He took off his clothes.
It's been funny to see how the world changes as the years go by. It would be fun to see how things change over the years.
That stinks of cheese. It smells like cheese.
Are they students? Are you students?
The quick brown fox jumps over a lazy dog. Eat some more soft French buns and drink some tea.
I know what you're up to. I know what you're planning.
Let's hope that that happens. Let's hope that that happens.
This morning, I was woken up by the telephone. The telephone woke me up this morning.


In [44]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4  # Define smoothing function
rouge_scores = []
for ref, hyp in zip(val_targets, val_predictions):
    rouge_scores.append(sentence_bleu(ref, hyp, smoothing_function=smoothie))

# Compute the average ROUGE score
average_rouge_score = sum(rouge_scores) / len(rouge_scores)

# Print the average ROUGE score
print("Average ROUGE score:", average_rouge_score)

Average ROUGE score: 0.02138867080699102


In [40]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.ter import ter, Tercom

# Compute BLEU score
references = val_targets  # List of reference paraphrases
hypotheses = val_predictions  # List of generated paraphrases
bleu_score = corpus_bleu([[ref] for ref in references], hypotheses)
print("BLEU score:", bleu_score)

# Compute TER score
ter_score = ter(hypotheses, references)
print("TER score:", ter_score)

# Compute Distinct n-grams
n = 2  # Specify the value of n for n-grams
distinct_ngrams = set()
for hypothesis in hypotheses:
    tokens = hypothesis.split()
    distinct_ngrams.update(nltk.ngrams(tokens, n))
distinct_ngrams_count = len(distinct_ngrams)
distinct_ngrams_score = distinct_ngrams_count / len(hypotheses)
print(f"Distinct {n}-grams score:", distinct_ngrams_score)


ModuleNotFoundError: No module named 'nltk.translate.ter'